In [ ]:
!pip install turicreate
!pip install numpy

# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [ ]:
import turicreate
import numpy as np

In [ ]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive/"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive')

Google Drive already mounted


In [ ]:
import os.path
import urllib.request
import tarfile
import zipfile
import gzip
from shutil import copy

def fetch_remote_datafile(filename, remote_url):
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
    return
  print("fetching " + filename + " from " + remote_url + "...")
  urllib.request.urlretrieve(remote_url, "./" + filename)

def cache_datafile_in_drive(filename):
  if os.path.isfile("./" + filename) == False:
    print("cannot cache " + filename + ", it is not in workspace")
    return
  
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile(data_drive_path + filename):
    print("" + filename + " has already been stored in Google Drive")
  else:
    print("copying " + filename + " to " + data_drive_path)
    copy("./" + filename, data_drive_path)
  

def load_datafile_from_drive(filename, remote_url=None):
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
  elif os.path.isfile(data_drive_path + filename):
    print("have " + filename + " in Google Drive, copying to workspace...")
    copy(data_drive_path + filename, ".")
  elif remote_url != None:
    fetch_remote_datafile(filename, remote_url)
  else:
    print("error: you need to manually download " + filename + " and put in drive")
    
def extract_datafile(filename, expected_extract_artifact=None):
  if expected_extract_artifact != None and (os.path.isfile(expected_extract_artifact) or os.path.isdir(expected_extract_artifact)):
    
    print("files in " + filename + " have already been extracted")
  elif os.path.isfile("./" + filename) == False:
    print("error: cannot extract " + filename + ", it is not in the workspace")
  else:
    extension = filename.split('.')[-1]
    if extension == "zip":
      print("extracting " + filename + "...")
      data_file = open(filename, "rb")
      z = zipfile.ZipFile(data_file)
      for name in z.namelist():
          print("    extracting file", name)
          z.extract(name, "./")
      data_file.close()
    elif extension == "gz":
      print("extracting " + filename + "...")
      if filename.split('.')[-2] == "tar":
        tar = tarfile.open(filename)
        tar.extractall()
        tar.close()
      else:
        data_zip_file = gzip.GzipFile(filename, 'rb')
        data = data_zip_file.read()
        data_zip_file.close()
        extracted_file = open('.'.join(filename.split('.')[0:-1]), 'wb')
        extracted_file.write(data)
        extracted_file.close()
    elif extension == "tar":
      print("extracting " + filename + "...")
      tar = tarfile.open(filename)
      tar.extractall()
      tar.close()
    elif extension == "csv":
      print("do not need to extract csv")
    else:
      print("cannot extract " + filename)
      
def load_cache_extract_datafile(filename, expected_extract_artifact=None, remote_url=None):
  load_datafile_from_drive(filename, remote_url)
  extract_datafile(filename, expected_extract_artifact)
  cache_datafile_in_drive(filename)
  
load_cache_extract_datafile("home_data.sframe.zip", "home_data.sframe", "https://d3c33hcgiwev3.cloudfront.net/00FsKOIoEemELQpo9cj5Ig_579152614f2b4a399ac90a939360749e_home_data.sframe.zip?Expires=1583366400&Signature=UOIwIuuT~0UT9V6UveCG9Vanabu3gWSsrwu63-U4VNNkahfkATwOcJ7ipdH9pEbfHYP8SW8UZFozRcBDk95Ygv4jN9tDl2dI3q8~AXs2K9GEGyhdsER~aTY4n8K6Vqhmba9KvFRyAIiFQF7jPy2K6LFGBHEBswUY0INusC-twkE_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")


already have home_data.sframe.zip in workspace
files in home_data.sframe.zip have already been extracted
home_data.sframe.zip has already been stored in Google Drive


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [ ]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [ ]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [ ]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.615693     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 0.804094     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [ ]:
model_all.coefficients.print_rows(18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049594 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910027  |  None  |
|   sqft_living    |  None | 24.420720982446056 |  None  |
| sqft_living_sqrt |  None | 350.0605533860446  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348975227  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [ ]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [ ]:
l1_penalty = np.logspace(1, 7, num=13)
l1_keys_values = {}
for i in l1_penalty:
  training_model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=i,verbose = False)
  predictions = training_model.predict(validation)
  RSS_for_validation = sum((validation['price'] - predictions)**2)
  l1_keys_values[i] = RSS_for_validation
print(l1_keys_values)

{10.0: 625766285142461.4, 31.622776601683793: 625766285362395.2, 100.0: 625766286057886.9, 316.22776601683796: 625766288257224.9, 1000.0: 625766295212185.9, 3162.2776601683795: 625766317206077.8, 10000.0: 625766386760661.5, 31622.776601683792: 625766606749281.4, 100000.0: 625767302791633.4, 316227.7660168379: 625769507643885.0, 1000000.0: 625776517727025.9, 3162277.6601683795: 625799062845466.9, 10000000.0: 625883719085424.5}


In [ ]:
print(min(l1_keys_values.items(), key=lambda x: x[1]))

(10.0, 625766285142461.4)


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [ ]:
model_for_best_l1 = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=10.0,verbose = False)
model_for_best_l1.coefficients.print_rows(18)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.967679031305   |  None  |
| bedrooms_square  |  None |  936.9933681932998   |  None  |
|    bathrooms     |  None |  25409.58893412067   |  None  |
|   sqft_living    |  None |  39.11513637970762   |  None  |
| sqft_living_sqrt |  None |  1124.6502128077204  |  None  |
|     sqft_lot     |  None | 0.003483618222989788 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140815  |  None  |
|      floors      |  None |  21204.335466950135  |  None  |
|  floors_square   |  None |  12915.52433607243   |  None  |
|    waterfront    |  None |  601905.5945452717   |  None  |
|       view       |  None |  93312.85731187185   |  None  |
|    condition     |  None |  6609.035712447212   |  None  |
|      grade       |  No

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [ ]:
list_fo_non_zeros = {}
for l1_penalty_n in l1_penalty_values:
  sparisty_model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty_n,verbose = False)
  sparisty_model.coefficients.print_rows(18)
  print('the l1 penalty number',l1_penalty_n)
  list_fo_non_zeros[l1_penalty_n] = sparisty_model.coefficients['value'].nnz()
  print('the numbers of nonzeros in model',sparisty_model.coefficients['value'].nnz())
print('List of all l1 penalty and it"s non-zeros values : \n ' ,list_fo_non_zeros)

+------------------+-------+-----------------------+--------+
|       name       | index |         value         | stderr |
+------------------+-------+-----------------------+--------+
|   (intercept)    |  None |   25090.917367212027  |  None  |
|     bedrooms     |  None |   7789.177061100561   |  None  |
| bedrooms_square  |  None |    847.559686942873   |  None  |
|    bathrooms     |  None |   25234.209194458632  |  None  |
|   sqft_living    |  None |   39.03944596356859   |  None  |
| sqft_living_sqrt |  None |   1117.3118955675664  |  None  |
|     sqft_lot     |  None | -0.025686118239919564 |  None  |
|  sqft_lot_sqrt   |  None |   143.98899196976998  |  None  |
|      floors      |  None |   20695.35923964388   |  None  |
|  floors_square   |  None |   12466.690650291855  |  None  |
|    waterfront    |  None |   568204.6445841152   |  None  |
|       view       |  None |   91066.94280879853   |  None  |
|    condition     |  None |   6360.780926249814   |  None  |
|      g

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [ ]:
narrow_range_l1_pen = {}
list_narrow_range_for_non_zeros = {}
for l1_penalty in l1_penalty_values :
  narrow_range_model = turicreate.linear_regression.create(training, target = 'price',features = all_features,validation_set=None,verbose=None,l2_penalty=0,l1_penalty=l1_penalty)
  narrow_range_predictions = narrow_range_model.predict(validation)
  narrow_range_RSS = sum((validation['price'] - narrow_range_predictions)**2)
  narrow_range_l1_pen[l1_penalty] = narrow_range_RSS
  list_narrow_range_for_non_zeros[l1_penalty] = narrow_range_model.coefficients['value'].nnz()
print(narrow_range_l1_pen)
print('List of all l1 penalty and it"s non-zeros values : \n ' ,list_narrow_range_for_non_zeros)

{2976351441.6313133: 966925692362086.5, 3019316638.9524155: 974019450084557.2, 3062281836.2735176: 981188367942451.0, 3105247033.5946198: 989328342459472.0, 3148212230.915722: 998783211265885.9, 3191177428.236824: 1008477167020096.5, 3234142625.557926: 1018298780553818.6, 3277107822.8790283: 1028247992205976.1, 3320073020.2001305: 1034616909232829.2, 3363038217.5212326: 1038554735941039.9, 3406003414.8423347: 1043237237871704.1, 3448968612.163437: 1046937488751713.4, 3491933809.484539: 1051147625612863.1, 3534899006.805641: 1055992735342999.4, 3577864204.126743: 1060799531763290.2, 3620829401.447845: 1065707689498229.6, 3663794598.768947: 1069464335425584.1, 3706759796.0900493: 1073504549585594.6, 3749724993.4111514: 1077632775581412.0, 3792690190.7322536: 1081867592324111.9}
List of all l1 penalty and it"s non-zeros values : 
  {2976351441.6313133: 10, 3019316638.9524155: 10, 3062281836.2735176: 10, 3105247033.5946198: 10, 3148212230.915722: 10, 3191177428.236824: 10, 3234142625.55792

In [ ]:
rss_for_l1_with_sparsity_seven_list ={}
for k,v in list_narrow_range_for_non_zeros.items():
  if v == 7:
    rss_for_l1_with_sparsity_seven_list[k] = narrow_range_l1_pen[k]
print(min(rss_for_l1_with_sparsity_seven_list.items(), key=lambda x: x[1]))
best_li_penalty_model = turicreate.linear_regression.create(training, target = 'price',features = all_features,validation_set=None,verbose=None,l2_penalty=0,l1_penalty=r[0])
print(best_li_penalty_model.coefficients.print_rows(18))

(3448968612.163437, 1046937488751713.4)
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19254432796 |  None  |
|     bedrooms     |  None | 661.7227177822556  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 15873.957259267987 |  None  |
|   sqft_living    |  None | 32.41022145125966  |  None  |
| sqft_living_sqrt |  None | 690.1147733133253  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

10